### Установка библиотек

In [ ]:
!pip install python-telegram-bot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513 kB 5.7 MB/s 
     |████████████████████████████████| 423 kB 52.7 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0, but you have tornado 6.2 which is incompatible.


In [ ]:
!pip install python-telegram-bot --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# перезапускаем среду выполнения

In [1]:
!pip install pymorphy2
!pip install stop_words
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 8.2 MB 38.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=843f01ad7ea8428deaa3f15dbbd7aa620c3306dfdadfc9e1bd5bc1c3d8001f36
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 9.9 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391583 sha256=81b4223c7308264cca2afa5b533ea1bd3f3dba78d4585a3a78a63d7187fc8574
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f

In [2]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
import datetime
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

### Блок обработки текста

In [3]:
morpher = MorphAnalyzer()
stop_words = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [ ]:
# удаляем слово "время" из множества stop_words
stop_words.discard('время')

In [4]:
def preprocess_txt(input_txt):
    txt = "".join(i for i in input_txt.strip() if i not in exclude).split()
    txt = [morpher.parse(i.lower())[0].normal_form for i in txt]
    txt = [i for i in txt if i not in stop_words and i != ""]
    return txt

In [57]:
# Пути к данным
path_otvety = '/content/drive/MyDrive/Otvety.txt'
path_prepared_answers = '/content/drive/MyDrive/prepared_answers.txt'
path_model = '/content/drive/MyDrive/ft_model'
path_index = '/content/drive/MyDrive/speaker.ann'
path_index_speaker = '/content/drive/MyDrive/index_speaker.pkl'
path_recepies = '/content/drive/MyDrive/recepies.csv'
path_index_recepies_ann = '/content/drive/MyDrive/recepies.ann'
path_index_recepies = '/content/drive/MyDrive/recepies.pkl'

In [ ]:
assert True
#Данные для режима болтанки
question = None
written = False

with open(path_prepared_answers, "w") as fout:
    with open(path_otvety, "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


0it [00:00, ?it/s]

In [ ]:
assert True

# Preprocess for models fitting
sentences = []
c = 0

with open(path_otvety, "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

In [ ]:
sentences = [i for i in sentences if len(i) > 2]
print('start train fasttext')
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5, workers=8)

start train fasttext


In [ ]:
modelFT.save(path_model)

In [19]:
modelFT = FastText.load(path_model)

In [20]:
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open(path_prepared_answers, "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save(path_index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


0it [00:00, ?it/s]

True

In [25]:
with open(path_index_speaker, "wb") as f:
    pickle.dump(index_map, f)

In [26]:
with open(path_index_speaker, 'rb') as f:
    index_map = pickle.load(f)

In [8]:
# Загрузка обученной модели
modelFT = FastText.load(path_model)
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load(path_index) 
ft_index.get_nns_by_vector(np.zeros(100), 2)

[7173, 52674]

In [9]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    return vector_ft / n_ft

In [85]:
# Тест модели для болтанки
vect_ft = embed_txt(preprocess_txt('привет'), {}, 1)
ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
index_map[ft_index_val[0]]

'Привет, тебя всегда приятно видеть!. \n'

##### Дообучение модели на втором наборе данных

In [ ]:
# Загрузка датасета с рецептами
rec_data = pd.read_csv('/content/drive/MyDrive/all_recepies_inter.csv', sep='\t')


rec_data['text'] = rec_data['name'] + " " + rec_data['Инструкции']
rec_data['text'] = rec_data['text'].apply(lambda x: preprocess_txt(str(x)))

In [ ]:
rec_data.to_csv(path_recepies, encoding='utf-8', index=False)

In [93]:
rec_data = pd.read_csv(path_recepies)
rec_data.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter,text
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая...","['рассольник', 'классический', 'перловка', 'со..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ...","['суп', 'пюре', 'белокочаный', 'капуста', 'нео..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква...","['постный', 'щи', 'квашеный', 'капуста', 'чест..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni...","['тюря', 'простой', 'суп', 'быстро', 'вкусно',..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u...","['фасолевый', 'суп', 'красный', 'фасоль', 'под..."


In [94]:
rec_data.shape

(27884, 11)

In [95]:
rec_data = rec_data[['name', 'Инструкции', 'text']][:10000]
rec_data.head()

,name,Инструкции,text
0,рассольник классический с перловкой и солеными...,Подготовить указанные ингредиенты для приготов...,"['рассольник', 'классический', 'перловка', 'со..."
1,Суп пюре из белокочаной капусты,"Необходимые ингредиенты\r\nНарезаем лук, морко...","['суп', 'пюре', 'белокочаный', 'капуста', 'нео..."
2,Постные щи из квашеной капусты,"Честно признаюсь, у меня не было репы на момен...","['постный', 'щи', 'квашеный', 'капуста', 'чест..."
3,Тюря- простой суп быстро и вкусно,"\r\nНачинаем мы приготовление тюри с того, что...","['тюря', 'простой', 'суп', 'быстро', 'вкусно',..."
4,Фасолевый суп из красной фасоли,Подготовить ингредиенты. Для приготовления суп...,"['фасолевый', 'суп', 'красный', 'фасоль', 'под..."


In [96]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [97]:
idxs = set(np.random.randint(0, len(index_map), len(rec_data)))

In [98]:
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]

In [99]:
positive_texts = [" ".join(val) for val in rec_data['text'].values]

In [100]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [101]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.3, stratify=labels,
                                                    random_state=13)

In [102]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [103]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.845665608819108

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)
np.mean(tfidf_vect.idf_)

9.56586926331154

In [105]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [106]:
ft_index_rec = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_rec = {}
counter = 0

for i in tqdm_notebook(range(len(rec_data))):
    n_ft = 0
    index_map_rec[counter] = (rec_data.loc[i, "name"], rec_data.loc[i, "Инструкции"])
    vector_ft = np.zeros(100)
    for word in rec_data.loc[i, "text"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_rec.add_item(counter, vector_ft)
    counter += 1

ft_index_rec.build(10)
ft_index_rec.save(path_index_recepies_ann)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/10000 [00:00<?, ?it/s]

True

In [107]:
with open(path_index_recepies, "wb") as f:
    pickle.dump(index_map_rec, f)

In [108]:
with open(path_index_recepies, 'rb') as f:
    index_map_rec = pickle.load(f)

In [110]:
ft_index_rec = annoy.AnnoyIndex(100, 'angular')
ft_index_rec.load(path_index_recepies_ann) 

midf = np.mean(tfidf_vect.idf_)
ft_index_rec.get_nns_by_vector(np.zeros(100), 3, include_distances=True)

([774, 786, 805], [1.4142135381698608, 1.4142135381698608, 1.4142135381698608])

In [111]:
# Тест модели для рецептов
vect_ft = embed_txt(preprocess_txt('Фасолевый суп из красной фасоли'), idfs, midf)
ft_index_rec_val = ft_index_rec.get_nns_by_vector(vect_ft, 3)
for item in ft_index_rec_val:
  name, ins = index_map_rec[item]
  print(f"блюдо: {name} инструкция: {ins}")

блюдо: Салат из огурцов и лука инструкция: Все овощи, кроме перчиков Чили, нарежьте. Смешайте в чашке.
Посолите.
Залейте салат из огурцов и лука уксусом. Помешайте.
Выложите зелёные перчики чили целиком.
Поставьте салат в холодильник на 10 минут. Подавайте охлаждённым.
блюдо: Котлеты из свеклы инструкция: 1. Свеклу очистить от кожуры, промыть и натереть на терке. Полученную массу разделать в виде котлет, обвалять в сухарях и обжарить в масле.
блюдо: Луковые котлеты инструкция: 1. В мелко нарубленный лук добавляем яйца, соль и муку. Лепим котлеты, обваливаем в муке, обжариваем на сковороде.
2. Выкладываем котлеты в кастрюльку, прослаивая слегка обжаренной нарезанной морковью, заливаем водой или сметанным соусом, тушим в духовке 30–40 минут.


### Функции бота

In [ ]:
def is_intent_time(input_txt):
  res = False
  if len(input_txt) < 6:
    for word in input_txt:
      if word == 'время' or word == 'час':
        res = True

  return res

In [ ]:
def is_intent_5k(input_txt):
  res = False
  if len(input_txt) < 6:
    for word in input_txt:
      if word == 'пятёрочка' or word == 'пятерочка':
        res = True

  return res  

In [ ]:
var_otv = {'yes': 1, 'no' : 2, 'dialog' : 0}

In [ ]:
def is_yes_no(input_txt):
  var = var_otv.get('dialog')
  if input_txt == 'да' or input_txt == 'Да':
    var = var_otv.get('yes')
  elif input_txt == 'нет' or  input_txt =='Нет':
    var =  var_otv.get('no')
  elif (len(input_txt.split()) > 1) and (input_txt.split()[0]=='да' or input_txt.split()[0] == 'Да'):
    var = var_otv.get('yes')

  return var

### Блок парсинга Пятёрочки

In [ ]:
from pathlib import Path
import time
import json
import requests

In [ ]:
class Parse5ka:
    headers = {"User-Agent": "ucheba"}

    def __init__(self, start_url: str):
        self.start_url = start_url

    def _get_response(self, url, *args, **kwargs) -> requests.Response:
        while True:
            response = requests.get(url, *args, **kwargs, headers=self.headers)
            if response.status_code in (200, 301, 304):
                return response
            time.sleep(1)

    def run(self):
        prod = dict()
        for product in self._parse(self.start_url):
            prod[product['name']] = product['current_prices'].get('price_promo__min')
        return prod
        
    def _parse(self, url):
        while url:
            response = self._get_response(url)
            data: dict = response.json()
            url = data.get("next")
            for product in data.get("results", []):
                yield product

In [ ]:
parser = Parse5ka("https://5ka.ru/api/v2/special_offers/")
product = parser.run()

In [ ]:
def price5k(prod = 'колбаса'):
  return [f'{key} - {val}р.' for key, val in product.items() if key.lower().find(prod.lower())!=-1]

### Блок соединения с телеграм ботом

In [ ]:
#@Aleksey_educ_bot#
#создание бота в Telegramm - BotFather 

In [ ]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
# Создадим соединение с ботом
updater = Updater("5393420496:AAFk_-R2VQH94tQUsaDqjYZC6twdwMltzPA", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher

In [ ]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день! Я учебный бот @Aleksey_educ_bot')
    update.message.reply_text('Могу просто поболтать')
    update.message.reply_text('Сказать рецепт блюда')
    update.message.reply_text('Показать скидки на продукты в Пятёрочке')
    update.message.reply_text('Подсказать наше время')

In [ ]:
intent = []
last_txt = ''

In [ ]:
def textMessage(update: Update, context: CallbackContext):
  otv_var = is_yes_no(update.message.text)
  global last_txt, price5k
  
  if otv_var == var_otv.get('yes'):
    if intent[0] == 'time':
      intent.clear()
      today = datetime.datetime.now()
      update.message.reply_text(f'у нас {today.strftime("%H.%M")}') 
    elif intent[0] == '5k':
      intent.clear()

      if len(update.message.text.split()) > 1:
        prod = price5k(" ".join(update.message.text.split()[1:]))
        if len(prod) > 0:
          for mes in prod:
            update.message.reply_text(mes)
        else:
            update.message.reply_text('Извините, на указанный продукт нет скидок')
      else:
        # колбаса
        for mes in price5k():
          update.message.reply_text(mes)
  
  elif otv_var == var_otv.get('no'):
    intent.clear()
    input_txt = last_txt
    last_txt = ''
  elif otv_var == var_otv.get('dialog'):
    input_txt = preprocess_txt(update.message.text)

  if otv_var == var_otv.get('dialog') and len(intent) == 1:  
    update.message.reply_text('выберете вариант ответа "да" или "нет"')
  elif otv_var == var_otv.get('dialog') and is_intent_time(input_txt):
    update.message.reply_text('вы хотите узнать который час?')
    intent.append('time')
    last_txt = input_txt
  elif otv_var == var_otv.get('dialog') and is_intent_5k(input_txt):
    update.message.reply_text('вы хотите узнать скидки в Пятёрочке? (после Да, можно указать продукт)')
    intent.append('5k')
    last_txt = input_txt
  elif otv_var == var_otv.get('dialog') or otv_var == var_otv.get('no'): 
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_rec_val.get_nns_by_vector(vect_ft, 3)
        for item in ft_index_shop_val:
            name, ins = index_map_rec[item]
            update.message.reply_text(f"блюдо: {name} инструкция: {ins}")
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.35:
        print(distances[0])
        update.message.reply_text("Извините, я вас не понимаю")
        return
    update.message.reply_text(index_map[ft_index_val[0]])

In [ ]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Start the Bot
updater.start_polling()
updater.idle()